In [2]:
#!pip install --upgrade tensorflow

In [3]:
import sys
import collections
import multiprocessing
from datetime import datetime as dt
import json
import os
import gc
import glob
import cv2
import seaborn as sns 
from matplotlib import pyplot as plt
from PIL import Image,ImageDraw,ImageFile
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import keras
import keras.backend as K
from tensorflow.keras.optimizers import Adam
from keras.layers import Dense, Flatten, Activation, Dropout, GlobalAveragePooling2D
from keras import optimizers, applications
from keras.models import Model, load_model
import tensorflow as tf
from keras.callbacks import Callback, ModelCheckpoint, LearningRateScheduler, TensorBoard, EarlyStopping
from tensorflow.keras.utils import Sequence,get_file,plot_model
from tqdm import tqdm_notebook as tqdm
tf.autograph.set_verbosity(0)
ImageFile.LOAD_TRUNCATED_IMAGES = True
%matplotlib inline

In [4]:
print(tf.__version__)

Les pièges à caméra (ou Wild Cams) permettent la collecte automatique de grandes quantités de données d'images. Les biologistes du monde entier utilisent les pièges à caméra pour surveiller la biodiversité et la densité de population des espèces animales.

Les données d'entraînement consistent en plus de 2,5 millions de séquences d'images collectées à l'aide de pièges à caméra placés dans plusieurs régions du monde et surtout en Afrique. Les séquences sont marquées d'un seul coup pour 266 groupes d'espèces différents, ou vides.

Nous allons suivre une approche de premier passage pour charger, comprendre et préparer les données. Nous utiliserons une approche d'apprentissage par transfert de caractéristiques pour entraîner un modèle Keras sur un sous-ensemble de données d'entraînement. Nous allons ensuite comment enregistrer votre modèle et l'enbarquer dans une API pour son utilisation en déploiement.

Le but de ce notebook est de catégoriser les espèces et de compter le nombre d'individus à travers les images afin de détecter une potentielle intrusion dans les cultures.

### Explication pour les fichier image

Tout d'abord, voyons quels types d'images sont disponibles.

In [5]:
TRAIN_DATA_PATH = '../input/iwildcam-2020-fgvc7/train/'
TEST_DATA_PATH = '../input/iwildcam-2020-fgvc7/test/'

train_jpeg = glob.glob(TRAIN_DATA_PATH + '*')
test_jpeg = glob.glob(TEST_DATA_PATH + '*')

print("number of train jpeg data:", len(train_jpeg))
print("number of test jpeg data:", len(test_jpeg))

### données d'entrainement

In [6]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(train_jpeg[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

### données de test :

In [7]:
fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(test_jpeg[:16]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize((480,270))
    plt.imshow(im)

Dans les pièges à caméra, les images sont prises en continu car elles sont capturées en rafales déclenchées par le mouvement. Par conséquent, le jeu de données contient également des séries d'images, et en plus des ID de l'image, les ID de la séquence sont attribués.nous utiliserons les ID de l'image pour charger l'image.
De plus, nous remarquerons que certaines des images sont en couleur et d'autres en noir et blanc. Cela est dû au fait que les images ont été prises de jour ou de nuit.

### Explication pour le fichier de métadonnées

Vérification de iwildcam2021_train_annotations.json
Les données d'annotation pour les données de formation nous sont fournies sous le nom de "iwildcam2021_train_annotations.json". Ce json suit le format COCO-CameraTraps avec des champs supplémentaires.

Si nous chargeons le json, nous pouvons constater qu'il contient trois valeurs clés.

In [8]:
with open('../input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json', encoding='utf-8') as json_file:
    train_annotations =json.load(json_file)
    
train_annotations.keys()

Dans la valeur des images, nous pouvons obtenir des données pour chaque image du wildcan. Wildcam prendra plusieurs images à la suite. La valeur de la clé 'seq_num_frames' est le nombre d'images, 'id' est l'identifiant de l'image, et 'seq_id' est l'identifiant associé à l'image prise séquentiellement. Ce 'seq_id' est le même que le 'Id' dans le fichier de soumission.

Extrayons les données correspondant au cliché de seq_id:302ad820-7d42-11eb-8fb5-0242ac1c0002.

In [9]:
train_annotations_seq = train_annotations["images"][94:104]
train_annotations_seq

Si nous voyons les images, nous pouvons constater qu'il s'agit d'une série d'images.

In [10]:
train_images_seq = [(TRAIN_DATA_PATH+item["id"]+'.jpg') for item in train_annotations_seq]
img_array = []
size = (480,270)

fig = plt.figure(figsize=(25, 16))
for i,im_path in enumerate(train_images_seq):
    ax = fig.add_subplot(4, 3, i+1, xticks=[], yticks=[])
    im = Image.open(im_path)
    im = im.resize(size)
    plt.imshow(im)
    
    img_array.append(im)

La valeur de la clé categories contient une liste d'espèces animales annotées. id 0 est vide. L'id va jusqu'à 571. 

In [11]:
df_categories = pd.DataFrame.from_records(train_annotations["categories"])
df_categories

In [12]:
df_categories[df_categories["id"]==115]

Chaque image d'entrainement a au moins une annotation associée. Les annotated category_ids sont dans la valeur de la clé "annotations".

In [13]:
train_annotations["annotations"][:10]

In [14]:
train_annotated_category = set([ annotation["category_id"] for annotation in train_annotations["annotations"]])
len(train_annotated_category)

### Vérification de iwildcam2021_test_information.json¶
Informations pour le jeu de données de test. Le format est similaire à iwildcam2021_train_annotations.json avec seulement la clé des images.

In [15]:
with open('../input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json', encoding='utf-8') as json_file:
    test_information =json.load(json_file)
    
test_information.keys()

In [16]:
test_information['images'][:2]

### Vérification de iwildcam2021_megadetector_results.json

Nous pouvons également utiliser le modèle MegaDetector de Microsoft AI for Earth. Ce modèle est entraîné à détecter les animaux, les personnes et les véhicules dans les images de caméras-pièges à l'aide de centaines de milliers de boîtes englobantes provenant de divers écosystèmes. Le modèle n'identifie pas les animaux, il ne fait que les trouver.

Des exemples de résultats de détection nous sont fournis dans "iwildcam2021_megadetector_results.json".

In [17]:
with open('../input/iwildcam-2020-fgvc7/iwildcam2020_megadetector_results.json', encoding='utf-8') as json_file:
    megadetector_results =json.load(json_file)
    
megadetector_results.keys()

Il y a trois données clé-valeur dans json.

Le résultat détecté est dans la valeur des images.

In [18]:
megadetector_results_df = pd.DataFrame(megadetector_results["images"])
megadetector_results_df.head()

In [19]:
def get_data(x):
    if x == []:
        return 0
    return len(x)

megadetector_results_df["detected_num"] = megadetector_results_df.loc[:, "detections"].map(get_data)

In [20]:
print(f"There are {len(megadetector_results_df)} detection data.")

In [21]:
megadetector_results_df.iloc[100]["detections"]

Nous pouvons voir le résultat.

In [22]:
def draw_bboxs(detections_list, im):
    """
    detections_list: list of set includes bbox.
    im: image read by Pillow.
    """
    
    for detection in detections_list:
        x1, y1,w_box, h_box = detection["bbox"]
        ymin,xmin,ymax, xmax=y1, x1, y1 + h_box, x1 + w_box
        draw = ImageDraw.Draw(im)
        
        imageWidth=im.size[0]
        imageHeight= im.size[1]
        (left, right, top, bottom) = (xmin * imageWidth, xmax * imageWidth,
                                      ymin * imageHeight, ymax * imageHeight)
        
        draw.line([(left, top), (left, bottom), (right, bottom),
               (right, top), (left, top)], width=4, fill='Red')

In [23]:
# Voyons la 80ème donnée de l'ensemble de données de formation.
data_index = 80

# chargement de la centième image
im = Image.open("../input/iwildcam-2020-fgvc7/train/" + megadetector_results_df.loc[data_index]['id'] + ".jpg")
im = im.resize((480,270))

# Écraser bbox
draw_bboxs(megadetector_results_df.loc[data_index]['detections'], im)

# affichage
plt.imshow(im)
plt.title(f"image {data_index} with bbox")

In [24]:
megadetector_results_df.loc[data_index]['detections'][0]["bbox"]

Il est également possible de recadrer la zone détectée comme ceci. Nous allons dans la suite enregistrer les images recadrées pour créer un jeu de données.

In [25]:
def get_crop_area(bbox, image_size):
    x1, y1,w_box, h_box = bbox
    ymin,xmin,ymax, xmax = y1, x1, y1 + h_box, x1 + w_box
    area = (xmin * image_size[0], ymin * image_size[1], 
            xmax * image_size[0], ymax * image_size[1])
    return area

crop_area = get_crop_area(megadetector_results_df.loc[data_index]['detections'][0]["bbox"], im.size)
im_croped = im.crop(crop_area)
plt.imshow(im_croped)

In [26]:
megadetector_results["info"]

In [27]:
megadetector_results["detection_categories"]

In [28]:
def get_data(x):
    if x == []:
        return 0
    return len(x)

megadetector_results_df["detected_num"] = megadetector_results_df.loc[:, "detections"].map(get_data)

Nous pouvons sélectionner les données d'entraînement en fonction du nombre d'animaux dans l'image.

In [29]:
megadetector_results_df = megadetector_results_df[megadetector_results_df['detected_num'] < 2]
megadetector_results_df = megadetector_results_df.rename(columns={'id': 'image_id'})
megadetector_results_df.head()

In [30]:
with open('../input/iwildcam-2020-fgvc7/iwildcam2020_train_annotations.json') as json_file:
    train_annotations_json = json.load(json_file)

In [31]:
train_annotations_json.keys()

In [32]:
df_annotations = pd.DataFrame(train_annotations_json["annotations"])
df_annotations.head()

In [33]:
df_images = pd.DataFrame(train_annotations_json["images"])
df_images.head()

In [34]:
df_categories = pd.DataFrame(train_annotations_json["categories"])
df_categories.head()

In [35]:
with open('../input/iwildcam-2020-fgvc7/iwildcam2020_test_information.json') as json_file:
    test_information_json = json.load(json_file)
test_information_json.keys()

In [36]:
df_images_test = pd.DataFrame(test_information_json["images"])
df_images_test.head()

In [37]:
df_categories_test = pd.DataFrame(test_information_json["categories"])
df_categories_test.head()

### DATA ANALYSIS

#### Distribution des données

On trace les données d'entrainement et de test dans la perspective suivante :

- Moment

- ID de la catégorie

- Emplacement

Quand les données ont-elles été prises ?

Comme les animaux peuvent modifier leur activité de temps en temps, nous voulons comprendre comment les données sont réparties dans le temps.

#### Aperçu mensuel

In [38]:
month_year = df_images['datetime'].map(lambda str: str[2:7])
labels_month_year = sorted(list(set(month_year)))

month_year_test = df_images_test['datetime'].map(lambda str: str[2:7])

In [39]:
fig, ax = plt.subplots(1,2, figsize=(30,7))
ax = plt.subplot(1,2,1)
ax = plt.title('Count of train data per month & year')
ax = sns.countplot(x=month_year, order=labels_month_year)
ax.set(xlabel='YY-mm', ylabel='count')
ax.set(ylim=(0,50000))

ax = plt.subplot(1,2,2)
ax = plt.title('Count of test data per month & year')

ax = sns.countplot(x=month_year_test, order=labels_month_year)
ax.set(xlabel='YY-mm', ylabel='count')
ax.set(ylim=(0,50000))

Les données commencent en 2013-01 mais il semble y avoir quelques manques. Par exemple, les données d'entrainement entre 2013-11 et 2014-02 sont manquantes.

Nous pouvons également constater que les données d'entrainement entre 2013-01 et 2013-07 sont plus riches que les autres points de temps.

Les données d'entrainement couvrent les données de test dans la perspective du point de temps.

In [40]:
labels_month = sorted(list(set(df_images['datetime'].map(lambda str: str[5:7]))))

In [41]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per month')
ax = sns.countplot(x=df_images['datetime'].map(lambda str: str[5:7] ), order=labels_month)
ax.set(xlabel='mm', ylabel='count')
ax.set(ylim=(0,55000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per month')
ax = sns.countplot(x=df_images_test['datetime'].map(lambda str: str[5:7] ), order=labels_month)
ax.set(xlabel='mm', ylabel='count')
ax.set(ylim=(0,55000))

Les données du train sont biaisées. En février, mars, juin et juillet, les données sont plus riches que les autres mois.

Les données d'entrainement couvrent les données de test dans la perspective du mois.

Les données pour novembre et décembre sont manquantes. L'hibernation des animaux pourrait en être la raison.

#### Aperçu par heure

In [42]:
train_taken_hour = df_images['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)
test_taken_hour = df_images_test['datetime'].map(lambda x: dt.strptime(x, '%Y-%m-%d %H:%M:%S.%f').hour)

In [43]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per hour')
ax = sns.countplot(x=train_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per hour')
ax = sns.countplot(x=test_taken_hour)
ax.set(xlabel='hour', ylabel='count')
ax.set(ylim=(0,20000))

Si nous décidons arbitrairement de la période diurne et nocturne, nous pouvons également calculer le nombre de données diurnes et nocturnes.

Par exemple, nous définissons "le jour" comme étant 6-17 heures et "la nuit" comme étant 18-5 heures.

In [44]:
train_taken_phase = train_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")
test_taken_phase = test_taken_hour.map(lambda x: "daytime" if x >= 6 and x < 18 else "night")

In [45]:
fig, ax = plt.subplots(1,2, figsize=(20,7))
ax = plt.subplot(1,2,1)
plt.title('Count of train data per phase')
ax = sns.countplot(x=train_taken_phase, order=["daytime", "night"])
ax.set(xlabel='phase', ylabel='count')
ax.set(ylim=(0,200000))

ax = plt.subplot(1,2,2)
plt.title('Count of test data per phase')
ax = sns.countplot(x=test_taken_phase, order=["daytime", "night"])
ax.set(xlabel='phase', ylabel='count')
ax.set(ylim=(0,200000))

In [46]:
#Memory savings
del train_taken_phase
del test_taken_phase
del train_taken_hour
del test_taken_hour
gc.collect()

### Combien de données y a-t-il par catégorie d'animal Id ?

In [47]:
fig = plt.figure(figsize=(30, 4))
labels_id = sorted(list(set(df_categories["id"])))
ax = sns.barplot(x="id", y="count",data=df_categories, order=labels_id)
ax.set(ylabel='count')
ax.set(ylim=(0,80000))
plt.title('distribution of count per id in train')

In [48]:
fig = plt.figure(figsize=(30, 4))
labels_id = sorted(list(set(df_categories["id"])))
ax = sns.barplot(x="id", y="count",data=df_categories_test, order=labels_id)
ax.set(ylabel='count')
ax.set(ylim=(0,80000))
plt.title('distribution of count per id in test')

La fréquence des données dans chaque identifiant est similaire pour les données d'entrainement et de test.

In [49]:
fig = plt.figure(figsize=(15, 9))
ax = sns.distplot(x=df_categories['count'][1:])
ax.set(ylim=(0,0.00005))
ax.set(xlabel='count')
plt.title('distribution of number of data per id zoomed')

La plupart des données sont inférieures à 5000 images.

Mais nous pouvons constater que le nombre d'identifiants spécifiques est très important et que les données sont biaisées.

#### Combien de données par emplacement ?

Nous sommes tenus de détecter les photographies prises à différents endroits, mais comment distribuer les données en fonction de l'endroit ?

In [50]:
labels_location_train = sorted(list(set(df_images['location'])))
labels_location_test = sorted(list(set(df_images_test['location'])))
labels_location = labels_location_train + labels_location_test

In [51]:
fig = plt.figure(figsize=(30, 4))
ax = sns.countplot(x=df_images['location'], order=labels_location)
ax.set(xlabel='location', ylabel='count')
plt.title('Count of train data per location')

In [52]:
fig = plt.figure(figsize=(30, 4))
ax = sns.countplot(x=df_images_test['location'], order=labels_location)
ax.set(xlabel='location', ylabel='count')
plt.title('Count of test data per location')

Les données d'entrainement et les données de test semblent avoir été prises à des endroits différents.

Le nombre de photos est très différent selon le lieu

#### Les différents types d'animaux sont-ils photographiés dans des endroits différents ?

Nous disposons de données relatives aux lieux et aux catégories d'animaux, ce qui nous permet d'étudier si différents types d'animaux sont photographiés à différents endroits.

In [53]:
loc_cat_df_test = pd.merge(df_images, df_annotations, left_on='id', right_on='image_id', how = "inner").loc[:,["location", "category_id", "image_id"]]
loc_cat_df_test.head()

In [54]:
loc_cat_dict = {}
for loc in set(loc_cat_df_test["location"]):
    loc_cat_dict[loc] = list(set(loc_cat_df_test[loc_cat_df_test["location"] == loc]["category_id"]))   
loc_cat_matrix = np.zeros([loc_cat_df_test["location"].max()+1, loc_cat_df_test["category_id"].max()+1])
loc_cat_matrix.shape

In [55]:
for loc in loc_cat_dict.keys():
    for cat in loc_cat_dict[loc]:
        loc_cat_matrix[loc, cat] = 1

In [56]:
fig = plt.figure(figsize=(15, 15))
ax = sns.heatmap(loc_cat_matrix)
ax.set(xlabel='category', ylabel='location')
plt.title('Relation between animal categories and locations')

C'est un peu difficile à voir, mais les colonnes sont les catégories d'animaux et les rangées sont les lieux. Comme des motifs similaires apparaissent dans le sens vertical, il semble que des espèces similaires puissent apparaître même dans des endroits différents.

#### Combien d'animaux dans chaque séquence de photos ?

Nous pouvons savoir combien d'animaux chaque séquence photographique par la valeur des annotations de train_annotations_json. La clé Count semble représenter le nombre d'animaux apparaissant dans chaque séquence photographique.

In [57]:
df_annotations[2429:2440]

In [58]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][2429:2440]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("../input/iwildcam-2020-fgvc7/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

Le nombre d'individus qui apparaissent en même temps varie-t-il selon l'espèce animale ?

In [59]:
c = collections.Counter(df_annotations.loc[:,"count"])
animals_in_pict = list(c.keys())
freq = list(c.values())
k = zip(animals_in_pict,freq)

In [60]:
animals_in_pict_df = pd.DataFrame(sorted(k),columns=['num_of_animals','freq'])

In [61]:
animals_in_pict_df.head()

Seul un nombre limité d'animaux est regroupé. 266 espèces sont prises en compte dans le jeu d'entraînement, mais seules 14 espèces ont été photographiées, avec plus de 10 animaux à la fois. Il s'avère qu'il existe une relation entre l'espèce animale et le nombre d'individus reflétés.

In [62]:
species_morethan_10 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 10].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_10)].loc[:,["id", "name"]]

Une espèce qui montre plus de 30 individus à la fois sur une photo est seulement une vache et un sanglier.

In [63]:
species_morethan_30 = list(set(df_annotations[df_annotations.loc[:,"count"] >= 30].loc[:,"category_id"]))
df_categories[df_categories.loc[:,"id"].isin(species_morethan_30)].loc[:,["id", "name"]]

In [64]:
df_annotations[(df_annotations.loc[:,"category_id"] == 2) & (df_annotations.loc[:,"count"] >= 30)].head()

In [65]:
fig = plt.figure(figsize=(25, 16))
#for i,im_path in enumerate(df_annotations[df_annotations.loc[:,"count"] == 3].loc[:,"image_id"][2429:2440]):
for i,im_path in enumerate(df_annotations.loc[:,"image_id"][149674:149678]):
    ax = fig.add_subplot(4, 4, i+1, xticks=[], yticks=[])
    im = Image.open("../input/iwildcam-2020-fgvc7/train/" + im_path + ".jpg")
    im = im.resize((480,270))
    plt.imshow(im)

In [66]:
df_annotations[(df_annotations.loc[:,"category_id"] == 71) & (df_annotations.loc[:,"count"] >= 30)].head()

In [67]:
jpeg ='../input/iwildcam-2020-fgvc7/train/8897a9f8-21bc-11ea-a13a-137349068a90.jpg'
im = Image.open(jpeg)
im = im.resize((480,270))
plt.imshow(im)

### Préparation des données et du modèle :

In [68]:
#suppression de 90% des données concernant la catégorie empty pour eviter de 'surprédire' cette dernière
#df_annotations = df_annotations.drop(df_annotations[df_annotations['category_id'] ==0].sample(frac=.9).index)


In [69]:
fig = plt.figure(figsize=(15, 4))
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
#ax.set(ylim=(0,80000))
plt.title('distribution of count per animals each data of train')

In [70]:
df_annotations.head()

In [71]:
print(f"On a {len(df_categories) - 1} spèces")

In [72]:
df_annotations.to_csv("train_annotations.csv", index=False)
df_images.to_csv("train_annotations_images.csv", index=False)
df_categories.to_csv("train_annotations_categories.csv", index=False)

In [73]:
df_images_test.to_csv("annotations_images_test.csv", index=False)
df_categories_test.to_csv("annotations_categories_test.csv", index=False)

In [74]:
train_anns_df = df_annotations[['image_id','category_id']]
train_img_df = df_images[['id', 'file_name']].rename(columns={'id':'image_id'})
df_train_file_cat = pd.merge(train_img_df, train_anns_df, on='image_id')
df_train_file_cat['category_id']=df_train_file_cat['category_id'].astype(str)
df_train_file_cat.head()

In [75]:
df_train_file_cat = pd.merge(megadetector_results_df, df_train_file_cat, on='image_id', how='inner')

fig = plt.figure(figsize=(15, 4))
ax = sns.countplot(x="detected_num", data=megadetector_results_df)
ax.set(ylabel='count')
ax.set(ylim=(0,80000))
plt.title('distribution of count per animals each data of train')

In [76]:
#Parameters

batch_size = 256
img_size = 96
lr = 0.001 
nb_classes = len(df_categories)
nb_epochs = 100

In [77]:
%%time

train_datagen=ImageDataGenerator(
    validation_split=0.25,
    horizontal_flip = True,    
    zoom_range = 0.3,
    width_shift_range = 0.3,
    height_shift_range=0.3
    )

train_generator=train_datagen.flow_from_dataframe(    
    dataframe=df_train_file_cat[:50000],    
    directory="../input/iwildcam-2020-fgvc7/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",    
    target_size=(img_size,img_size))

test_datagen = ImageDataGenerator()

valid_generator=test_datagen.flow_from_dataframe(  
    dataframe=df_train_file_cat[50000:],    
    directory="../input/iwildcam-2020-fgvc7/train",
    x_col="file_name",
    y_col="category_id",
    batch_size=batch_size,
    shuffle=True,
    classes = [ str(i) for i in range(nb_classes)],
    class_mode="categorical",  
    target_size=(img_size,img_size))

In [78]:
def get_model():
    K.clear_session()
    tf.random.set_seed(2022)
    base_model =  tf.keras.applications.EfficientNetB3(weights='imagenet', include_top=False, pooling='avg', input_shape=(img_size, img_size, 3))
    x = base_model.output
    predictions = Dense(nb_classes, activation="softmax")(x)
    return Model(inputs=base_model.input, outputs=predictions)

model = get_model()
model.compile(Adam(learning_rate=lr, decay=1e-6),loss='categorical_crossentropy',metrics=['accuracy'])

In [79]:
#plot_model(model, show_shapes=True)

In [80]:
early = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=1, mode='auto')

In [81]:
%%time
history = model.fit_generator(generator=train_generator,  
                                    steps_per_epoch=5,
                                    validation_data=valid_generator, 
                                    validation_steps=2,
                                    epochs=nb_epochs,
                                    verbose=1,
                                    callbacks = [early])

In [82]:
history_df = pd.DataFrame(history.history)
history_df[['loss', 'val_loss']].plot()
history_df[['accuracy', 'val_accuracy']].plot()

In [83]:
sam_sub_df = pd.read_csv('../input/iwildcam-2020-fgvc7/sample_submission.csv')
sam_sub_df["file_name"] = sam_sub_df["Id"].map(lambda str : str + ".jpg")
sam_sub_df.head()

In [84]:
#%%time
#test_generator = test_datagen.flow_from_dataframe(      
    
        #dataframe=sam_sub_df,    
        #directory = "../input/iwildcam-2020-fgvc7/test",    
        #x_col="file_name",
        #target_size = (img_size,img_size),
        #batch_size = 1,
        #classes = [ str(i) for i in range(nb_classes)],
        #shuffle = False,
        #class_mode = None
        #)

In [85]:
#%%time
#predict=model.predict_generator(test_generator, steps = len(test_generator.filenames))
#predicted_class_indices=np.argmax(predict,axis=1)
#sam_sub_df["Category"] = predicted_class_indices
#sam_sub_df = sam_sub_df.loc[:,["Id", "Category"]]
#sam_sub_df.to_csv("test_result.csv",index=False)

In [86]:
#test_generator.reset()

# Evaluate on Validation data
#scores = model.evaluate(test_generator)
#print("%s%s: %.2f%%" % ("evaluate ",model.metrics_names[1], scores[1]*100))

#scores = model.evaluate_generator(test_generator)
#print("%s%s: %.2f%%" % ("evaluate_generator ",model.metrics_names[1], scores[1]*100))
#print("%s%s: %.2f%%" % ("loss=", scores[0]*100))

In [87]:
import gc
del train_datagen, train_generator
gc.collect()

In [88]:
sam_sub_df = pd.read_csv('../input/iwildcam-2020-fgvc7/sample_submission.csv')
sam_sub_df["file_name"] = sam_sub_df["Id"].map(lambda str : str + ".jpg")
sam_sub_df.head()

In [89]:
model.save('cultura_model.h5')

In [90]:
!ls

### Téléchargement du modèle :

In [98]:
!ls

In [99]:
from IPython.display import FileLink
FileLink(r'cultura_model.h5')